In [32]:
import pandas as pd
import numpy as np
from datetime import date
import re
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings

import feature_generation as fg
import imputation as imp
import evaluation as eva
import train_test_split as tts
import clf_define as clfd
import data_util as util

In [33]:
data = util.read_csv_data(r"projects_2012_2013.csv", {}, parse_dates=['date_posted', 'datefullyfunded'])

First of all, we will like to define our features and classifiers in the dataframe. In order to maintain the as much patterns as possible, we will only drop the columns with ids which include too much distinguish categories (more than 51) and information like logitude and latitude which can be categorized better using city and state. The number 51 is selected due to the number of state categories.

In [34]:
preserved_columns = []
for column in data.columns:
    if (data[column].dtype=='O') and (len(data[column].unique())<=51):
        preserved_columns.append(column)

In [35]:
print(preserved_columns)

['school_state', 'school_metro', 'school_charter', 'school_magnet', 'teacher_prefix', 'primary_focus_subject', 'primary_focus_area', 'secondary_focus_subject', 'secondary_focus_area', 'resource_type', 'poverty_level', 'grade_level', 'eligible_double_your_impact_match']


In [36]:
continuous_features = ['total_price_including_optional_support', 'students_reached']
categorical_features = set(preserved_columns) - set(continuous_features)

In [37]:
print(categorical_features)

{'school_metro', 'grade_level', 'secondary_focus_area', 'poverty_level', 'secondary_focus_subject', 'eligible_double_your_impact_match', 'school_state', 'teacher_prefix', 'school_magnet', 'school_charter', 'primary_focus_subject', 'primary_focus_area', 'resource_type'}


In [38]:
data[continuous_features + list(categorical_features)].head()

,total_price_including_optional_support,students_reached,school_metro,grade_level,secondary_focus_area,poverty_level,secondary_focus_subject,eligible_double_your_impact_match,school_state,teacher_prefix,school_magnet,school_charter,primary_focus_subject,primary_focus_area,resource_type
0,1498.61,31.0,urban,Grades PreK-2,Music & The Arts,highest poverty,Visual Arts,f,IL,Mrs.,f,f,Mathematics,Math & Science,Supplies
1,282.47,28.0,urban,Grades 3-5,Literacy & Language,highest poverty,Literature & Writing,t,CA,Mrs.,f,f,Civics & Government,History & Civics,Books
2,1012.38,56.0,urban,Grades 3-5,History & Civics,high poverty,Social Sciences,f,CA,Ms.,f,f,Literacy,Literacy & Language,Technology
3,175.33,23.0,urban,Grades PreK-2,NaN,high poverty,NaN,f,NY,Ms.,t,f,Literacy,Literacy & Language,Books
4,3591.11,150.0,suburban,Grades PreK-2,Literacy & Language,high poverty,Literature & Writing,f,NY,Mrs.,f,f,Literacy,Literacy & Language,Technology


We label 1 to the project which are not funded in 60 days right after they are posted, and label 0 to those who are funded in 60 days right after it is posted.

In [39]:
data['duration'] = data["datefullyfunded"] - data["date_posted"]
data['classifier'] =  np.where(data['duration'] > pd.Timedelta('60 days'), 1, 0)

In [40]:
classifier = "classifier"
data["classifier"].head()

0    0
1    0
2    1
3    0
4    1
Name: classifier, dtype: int32

We then move on to split the dataframe to train and test sets with rolling windows stratefy in this case. I split the dataframe with time unit of half years cutoff which is at date 6/30 and 12/31. For every training data set, there will remain 60 days gap for the result (label) of the outcome (classifier) to be observed. The testing set will be set as half year (might be shorter in the last test set) right after the gap.

In [41]:
half_year_stamp = [(1,1), (7,1)]
start_date = pd.Timestamp('2012-01-01 00:00:00')
end_date = pd.Timestamp('2014-01-01 00:00:00')

time_cutoff lists includes the start date, one day after the end dates and other cutoff dates, time interval will be defined as the date between them (including the lower but not include the upper bound).

In [42]:
time_cutoffs = tts.gen_time_cuts(start_date, end_date, half_year_stamp)
print(sorted(time_cutoffs))

[Timestamp('2012-01-01 00:00:00'), Timestamp('2012-07-01 00:00:00'), Timestamp('2013-01-01 00:00:00'), Timestamp('2013-07-01 00:00:00'), Timestamp('2014-01-01 00:00:00')]


I use define_time_variables to create the indicators for rolling window split in this case, taking waiting times (60 days) into account. All train dataframe starts in 2012-01-01 but the testing data starts at 2012-07-01 + (60 days), 2013-01-01 + (60 days), 2013-07-01 + (60 days) respectively.

In [43]:
data = tts.define_time_variables(data, pd.Timedelta('183 days'), pd.Timedelta('60 days'), "date_posted", "datefullyfunded", start_date, end_date, half_year_stamp)

In [44]:
data[data["tmp_label0"] == "train"]["date_posted"].sort_values(ascending=True).head()

69180   2012-01-01
32005   2012-01-01
38220   2012-01-01
74297   2012-01-01
21950   2012-01-01
Name: date_posted, dtype: datetime64[ns]

In [45]:
data[data["tmp_label0"] == "test"]["date_posted"].sort_values(ascending=True).head()

16332    2012-08-30
60107    2012-08-30
71880    2012-08-30
107093   2012-08-30
107156   2012-08-30
Name: date_posted, dtype: datetime64[ns]

In [46]:
data[data["tmp_label1"] == "train"]["date_posted"].sort_values(ascending=True).head()

72487    2012-01-01
63421    2012-01-01
117377   2012-01-01
62941    2012-01-01
117198   2012-01-01
Name: date_posted, dtype: datetime64[ns]

In [47]:
data[data["tmp_label1"] == "test"]["date_posted"].sort_values(ascending=True).head()

121525   2013-03-02
37673    2013-03-02
3158     2013-03-02
108509   2013-03-02
30275    2013-03-02
Name: date_posted, dtype: datetime64[ns]

In [48]:
data[data["tmp_label2"] == "train"]["date_posted"].sort_values(ascending=True).head()

63675    2012-01-01
114946   2012-01-01
48094    2012-01-01
50966    2012-01-01
107459   2012-01-01
Name: date_posted, dtype: datetime64[ns]

In [49]:
data[data["tmp_label2"] == "test"]["date_posted"].sort_values(ascending=True).head()

66491    2013-08-30
121728   2013-08-30
15134    2013-08-30
111442   2013-08-30
93247    2013-08-30
Name: date_posted, dtype: datetime64[ns]

The data is then be splitted according to the time_split_indicators

In [50]:
time_split_indicators = ["tmp_label0", "tmp_label1", "tmp_label2"]

In [51]:
data_dict = tts.rolling_window_split(data, time_split_indicators, list(categorical_features) + continuous_features, classifier)

We than move on to the imputation of missing values in the data of each sub set, we used the trained infromation in the training dataframe is used it in the transformation in the corresponding test dataframe. We will used different strategies with different missing values in the dataframe. For the categorical variables, we will use a "unknown" category to replace the unknown values. (Using fill_unknown) For the continuous variable, we will use the group mean of the training data to impute both training and testing dataframe. (Using fill_na_mean) 

We can observe several missing values in the given dataframe after rolling window split, we do not want to include classifier in the imputation since labeling unknown classification will be problematic with distribution taht we are not sure.

In [52]:
for idx, dat in data_dict.items():
    train_X, train_y, test_X, test_y = dat
    print("missing in training " + idx)
    print(imp.summarize_missing_values(train_X))
    print("missing in testing " + idx)
    print(imp.summarize_missing_values(test_X))
    print("           ")

missing in training tmp_label2
{'school_metro': (9412, 80809), 'grade_level': (3, 80809), 'poverty_level': (0, 80809), 'secondary_focus_subject': (26340, 80809), 'students_reached': (59, 80809), 'school_state': (0, 80809), 'total_price_including_optional_support': (0, 80809), 'teacher_prefix': (0, 80809), 'school_magnet': (0, 80809), 'school_charter': (0, 80809), 'primary_focus_area': (15, 80809), 'resource_type': (17, 80809), 'primary_focus_subject': (15, 80809), 'secondary_focus_area': (26340, 80809), 'eligible_double_your_impact_match': (0, 80809)}
missing in testing tmp_label2
{'school_metro': (4250, 32994), 'grade_level': (0, 32994), 'poverty_level': (0, 32994), 'secondary_focus_subject': (10442, 32994), 'students_reached': (0, 32994), 'school_state': (0, 32994), 'total_price_including_optional_support': (0, 32994), 'teacher_prefix': (0, 32994), 'school_magnet': (0, 32994), 'school_charter': (0, 32994), 'primary_focus_area': (0, 32994), 'resource_type': (0, 32994), 'primary_focus_

In [53]:
imp_data_dict = {}
for idx, dat in data_dict.items():
    train_X, train_y, test_X, test_y = dat
    train_X, test_X = imp.fill_na_mean(train_X, test_X, continuous_features)
    train_X, test_X = imp.fill_unknown(train_X, test_X, categorical_features)
    imp_data_dict[idx] = [train_X, train_y, test_X, test_y]

We can observe that after the imputation the new_data_dict contains no missing values in the training and testing subsets

In [54]:
for idx, dat in imp_data_dict.items():
    train_X, train_y, test_X, test_y = dat
    print("missing in training " + idx)
    print(imp.summarize_missing_values(train_X))
    print("missing in testing " + idx)
    print(imp.summarize_missing_values(test_X))
    print("           ")

missing in training tmp_label0
{'school_metro': (0, 26386), 'grade_level': (0, 26386), 'poverty_level': (0, 26386), 'secondary_focus_subject': (0, 26386), 'students_reached': (0, 26386), 'school_state': (0, 26386), 'total_price_including_optional_support': (0, 26386), 'teacher_prefix': (0, 26386), 'school_magnet': (0, 26386), 'school_charter': (0, 26386), 'primary_focus_area': (0, 26386), 'resource_type': (0, 26386), 'primary_focus_subject': (0, 26386), 'secondary_focus_area': (0, 26386), 'eligible_double_your_impact_match': (0, 26386)}
missing in testing tmp_label0
{'school_metro': (0, 33357), 'grade_level': (0, 33357), 'poverty_level': (0, 33357), 'secondary_focus_subject': (0, 33357), 'students_reached': (0, 33357), 'school_state': (0, 33357), 'total_price_including_optional_support': (0, 33357), 'teacher_prefix': (0, 33357), 'school_magnet': (0, 33357), 'school_charter': (0, 33357), 'primary_focus_area': (0, 33357), 'resource_type': (0, 33357), 'primary_focus_subject': (0, 33357), 

After I impute the missing values, I move on to feature generations. In this process, the features generated in the training dataframe have to be consistant to the features in testing dataframe. All categorical variables in the selected features will be transformed to binary features and all continuous variables will be transformed to scaled continuous using MaxMinScale

In [55]:
ft_data_dict = {}
for idx, dat in imp_data_dict.items():
    train_X, train_y, test_X, test_y = dat
    train_X, test_X = fg.min_max_transformation(train_X, test_X, continuous_features)
    ft_data_dict[idx] = [train_X, train_y, test_X, test_y]

In [56]:
ft_data_dict2 = {}
for idx, dat in ft_data_dict.items():
    train_X, train_y, test_X, test_y = dat
    for cat_column in categorical_features:
        train_X, test_X = fg.category_to_binary(train_X, test_X, cat_column, 3)
    ft_data_dict2[idx] = [train_X, train_y, test_X, test_y]

We then define the final dictionary of data using for machine learning pipeline as ft_data_dict2

In [57]:
data_dict = ft_data_dict2

In this case, I will select the largest training subset with its corresponding testing subset after the imputation and feature generation. Best performed random forest model will also be used to determine the riskiest 5% cases for the following assignment

In [58]:
X_train, y_train, X_test, y_test = data_dict["tmp_label2"][0], data_dict["tmp_label2"][1], data_dict["tmp_label2"][2], data_dict["tmp_label2"][3]
rand = clfd.RandomForestClassifier()
rand.set_params(**{'min_samples_split': 10, 'n_jobs': 2, 'class_weight': None, 'random_state': None, 'min_impurity_decrease': 0.0, 'oob_score': False, 'n_estimators': 100, 'verbose': 0, 'criterion': 'gini', 'max_leaf_nodes': None, 'max_depth': 50, 'min_samples_leaf': 1, 'min_impurity_split': None, 'warm_start': False, 'min_weight_fraction_leaf': 0.0, 'max_features': 'sqrt', 'bootstrap': True})
model = rand.fit(X_train, y_train)
y_predp = model.predict_proba(X_test)

In [59]:
X_train.to_pickle("test_k")

In [60]:
X_test = X_test.reset_index(drop=True)
y_predp = pd.DataFrame(y_predp[:,1], columns=["y_predp"])
y_predp = y_predp.reset_index(drop=True)
full = X_test.join(y_predp)

In [61]:
sub_n = 0.05 * full.shape[0] #sub_n = 1650
sub_df = full.sort_values(by="y_predp", ascending=False)[:1650]
sub_df.drop(["y_predp"], axis=1)
sub_df = sub_df.reset_index(drop=True)

In [62]:
sub_df.to_pickle("sub_test_k")

We finally get the subset with only 5% of the testing cases which are the most riskiest not to be funded in 60 days